In [1]:
%run 'Load & Clean Rumor'.ipynb

Exception: File `"'Load.py"` not found.

In [2]:
rumor_dfs_claim = []
rumor_dfs_claim.append(p.DataFrame(rumor_dfs[0].claim))
rumor_dfs_claim.append(p.DataFrame(rumor_dfs[1].claim))
rumor_dfs_claim.append(p.DataFrame(rumor_dfs[2].claim))
rumor_dfs_claim[0]['label'] = rumor_dfs[0].claim_label
rumor_dfs_claim[1]['label'] = rumor_dfs[1].claim_label
rumor_dfs_claim[2]['label'] = rumor_dfs[2].claim_label

In [3]:
print(rumor_dfs_claim[0].label.value_counts(), rumor_dfs_claim[1].label.value_counts(), rumor_dfs_claim[2].label.value_counts())

Unverified    857
TRUE          737
FALSE         550
Name: label, dtype: int64 pants-fire     1110
false           731
barely-true     460
half-true       244
mostly-true     207
true            171
Name: label, dtype: int64 false           8765
mfalse          3319
mixture         2618
true            1480
mtrue            487
undetermined     151
legend            42
Name: label, dtype: int64


In [4]:
X_train = [] 
X_test = [] 
y_train = [] 
y_test = []

from sklearn.model_selection import train_test_split

for i in range(0,3):
    X_tr, X_te, y_tr, y_te = train_test_split(
         rumor_dfs_claim[i].drop('label', axis=1), rumor_dfs_claim[i].label,
        test_size=0.33, random_state=42)

    X_train.append(X_tr)
    X_test.append(X_te)
    y_train.append(y_tr)
    y_test.append(y_te)

In [5]:
import ktrain 
from ktrain import text 
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500)
trn = t.preprocess_train(X_train[0].claim.tolist(), y_train[0].values.tolist())
val = t.preprocess_test(X_test[0].claim.tolist(), y_test[0].values.tolist())
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

preprocessing train...
language: en
train sequence lengths:
	mean : 11
	95percentile : 18
	99percentile : 24


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 11
	95percentile : 18
	99percentile : 21


In [ ]:
learner.lr_find(show_plot=True, max_epochs=2)

simulating training for different learning rates... this may take a few moments...
Train for 239 steps
Epoch 1/2
  1/239 [..............................] - ETA: 11:37:22 - loss: 1.1999 - accuracy: 0.1667

In [ ]:
learner.fit_onecycle(8e-5, 4)
learner.validate(class_names=t.get_classes())
learner.view_top_losses(n=25, preproc=t)



begin training using onecycle policy with max lr of 8e-05...
Train for 240 steps, validate for 23 steps
Epoch 1/4
  2/240 [..............................] - ETA: 10:01:53 - loss: 1.2395 - accuracy: 0.3333    

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.predict('Trump is our savior')
predictor.explain('Trump is our savior')

predictor.save('/rumor_predictor')
reloaded_predictor = ktrain.load_predictor('/tmp/my_20newsgroup_predictor')
reloaded_predictor.predict('Trump is our savior')
reloaded_predictor.predict_proba('Trump is our savior')
reloaded_predictor.get_classes()